In [1]:
import warnings
import os
import sys

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

In [2]:
from src.main.models.bidirLSMT import BidirectionalLSTM
from src.main.utilities.utils import clean_text, stop_words_removal, get_dataset
from sklearn.model_selection import train_test_split
from keras_tuner import RandomSearch
from config.config import RESULTS_DIRECTORY
from config.config import TENSORBOARD_LOGS
import keras_tuner as kt
from tensorflow import keras

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\franc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# define model and pipeline
bidirLSTM = BidirectionalLSTM()
bidirLSTM.pipeline = [clean_text, stop_words_removal]
# import dataset and apply pipeline
inputs, targets = get_dataset(one_hot=True)

inputs = bidirLSTM.run_pipeline(inputs, save=True)

# stratified train test split
x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=42, stratify=targets)

In [4]:
                                        
# Random Grid Search

tuner = kt.RandomSearch( 
    bidirLSTM,
    objective='val_accuracy',
    max_trials=60,
    directory=RESULTS_DIRECTORY.format(repr(bidirLSTM)),
    project_name= repr(bidirLSTM),
)

# Callbacks: Early stopping and TensorBoard profiling
callbacks = [
    keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True),
    keras.callbacks.TensorBoard(TENSORBOARD_LOGS.format("convolutional"))
]

# Search for the best hyperparameters: model selection
tuner.search(x_train, y_train, callbacks=callbacks, validation_split=0.15)

Reloading Tuner from d:\Magistrale\HLT\Progetto\HLT-News-Category\results\BidirectionalLSTM\BidirectionalLSTM\tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
10000             |1000              |vocab_size
100               |100               |embedding_dim
100               |90                |max_sequence_length
128               |128               |lstm_units
0.1               |0.02              |learning_rate


Epoch 1/2


Traceback (most recent call last):
  File "d:\miniconda3\envs\HLT\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "d:\miniconda3\envs\HLT\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\miniconda3\envs\HLT\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\miniconda3\envs\HLT\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Magistrale\HLT\Progetto\HL

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "d:\miniconda3\envs\HLT\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "d:\miniconda3\envs\HLT\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\miniconda3\envs\HLT\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\miniconda3\envs\HLT\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
              ^^^^^^^^^^^^^^^

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def tokenize(sequences):
    # Tokenization
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sequences)
    sequences_as_integers = tokenizer.texts_to_sequences(sequences)

    # Padding
    max_sequence_length = 100  # Choose an appropriate value based on your data
    return pad_sequences(sequences_as_integers, maxlen=max_sequence_length)

In [ ]:
x_train = tokenize(x_train)
x_test = tokenize(x_test)

In [ ]:
bidirLSTM.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, None, 128)      │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,183,941 (4.52 MB)

 Trainable params: 1,183,941 (4.52 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
bidirLSTM.fit(x_train, y_train, batch_size=100000, epochs=1, validation_data=(x_test, y_test), verbose=1)

InvalidArgumentError: Graph execution error:

Detected at node sequential_1/embedding_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "d:\miniconda3\envs\HLT\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "d:\miniconda3\envs\HLT\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "d:\miniconda3\envs\HLT\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "d:\miniconda3\envs\HLT\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "d:\miniconda3\envs\HLT\Lib\asyncio\base_events.py", line 641, in run_forever

  File "d:\miniconda3\envs\HLT\Lib\asyncio\base_events.py", line 1987, in _run_once

  File "d:\miniconda3\envs\HLT\Lib\asyncio\events.py", line 88, in _run

  File "d:\miniconda3\envs\HLT\Lib\site-packages\ipykernel\kernelbase.py", line 545, in dispatch_queue

  File "d:

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {config.TENSORBOARD_LOGS.format(repr(bidirLSTM))} --port=6969

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
bidirLSTM.evaluate(x_test, y_test)

677/677 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - accuracy: 0.3981 - loss: 1.7121


[1.7013485431671143, 0.403362900018692]

In [ ]:
from src.main.utilities import plotting

y_pred = bidirLSTM.predict(x_test)
results = plotting.compute_performance(y_test, y_pred)
plotting.print_performance(results)


677/677 ━━━━━━━━━━━━━━━━━━━━ 23s 34ms/step


ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets